<a href="https://colab.research.google.com/github/jaredmontierth/summarizer/blob/main/fine_tuning_bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets 

In [ ]:
!pip install rouge.score nltk py7zr

In [ ]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = load_dataset('argilla/news-summary')
data.save_to_disk('/content/argilla')


In [ ]:
data = load_from_disk("/content/argilla")
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

<ipython-input-5-4cf07fc0a26c>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [ ]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [ ]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'dialogue', 'summary'])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
#sample the data
train_sample = tokenize_data['train'].shuffle(seed=123).select(range(1000))
validation_sample = tokenize_data['validation'].shuffle(seed=123).select(range(500))
test_sample = tokenize_data['test'].shuffle(seed=123).select(range(200))

In [ ]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [ ]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [ ]:
batch_size = 4

In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_rouge(pred):
  predictions, labels = pred
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}
  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)
  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'news-summary',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )

In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
!nvidia-smi

Mon Apr 17 18:37:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    31W /  70W |   9165MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.196900,0.378427,50.331900,25.029800,40.827500,40.818100,26.448000
2,0.140700,0.420369,50.969000,25.017100,40.841100,40.808400,28.684000
3,0.079200,0.492640,51.085700,25.201100,40.919300,40.864700,29.162000


TrainOutput(global_step=1500, training_loss=0.13892815907796224, metrics={'train_runtime': 1752.3723, 'train_samples_per_second': 1.712, 'train_steps_per_second': 0.856, 'total_flos': 3250656903168000.0, 'train_loss': 0.13892815907796224, 'epoch': 3.0})

In [ ]:
article = """
First lady Jill Biden's walk-back of her suggestion that runner-up Iowa should join NCAA women's basketball champion LSU for a visit to the White House didn't sit well with Tigers star Angel Reese.
Prompted by a discussion of Biden's comments during her Wednesday appearance on "The Paper Route Podcast," Reese said the Tigers should celebrate their title with former President Barack Obama and former first lady Michelle Obama rather than Joe and Jill Biden.
Jill Biden, at an appearance in Denver on Monday, had praised Iowa's sportsmanship and congratulated both teams. She also said that as part of the longstanding tradition of having champions visit the White House, Iowa should come as well "because they played such a good game."
The Tigers defeated Iowa 102-85 for the title in Dallas on Sunday.
EDITOR'S PICKS
Clark: Reese flak unfair, WH not for 'runner-ups'
1hM.A. Voepel
White House clarifies LSU women's invitation
2h
'In my moment': LSU star Reese explains gesture
3dM.A. Voepel
Reese on Monday called Jill Biden's suggestion "a joke."
Joe Biden invited LSU and men's champion UConn to the White House on Tuesday with no mention of Iowa. Vanessa Valdivia, a spokesperson for Jill Biden, said the first lady had meant no disrespect to LSU and that her comments were intended to applaud the historic game and all women athletes.
"I'm not going to lie to you, I don't accept the apology because of, you said what you said. I said what I said. And like, you can't go back on certain things that you say," Reese told podcast hosts Brandon Marshall and Ashley Nicole Moss.
"I mean, you felt like they should've came because of sportsmanship, right?" Reese added. "They can have that spotlight. We'll go to the Obamas. We'll see Michelle. We'll see Barack."
Some social media commenters noted the racial dynamics involved, saying that only winners should be rewarded with a White House visit and that hosting both teams would detract from the achievement by LSU's team, which is predominantly Black. The Iowa team is largely white. Others noted the important role of Black women in Democratic Party politics.
Following LSU's victory, coach Kim Mulkey said she would go to the White House if invited. Reese said Wednesday she was uncertain if she would go.
Reese faced criticism on social media for waving her hand in front of Iowa star Caitlin Clark's face while staring down Clark during the game. Clark, The Associated Press Player of the Year, made a similar gesture to no one in particular during Iowa's victory over Louisville in the Elite Eight.
Reese said she didn't think LSU, had it lost to Iowa, would have gotten the same praise from Jill Biden as the Hawkeyes did.
"If we were to lose, we would not be getting invited to the White House," she added. "I remember she made a comment about both teams should be invited because of sportsmanship. And I'm like, 'Are you saying that because of what I did?' Stuff like that, it bothers me because you are a woman at the end of the day. White, Black, it doesn't matter, you're a woman, you're supposed to be standing behind us before anything."

"""

In [ ]:
model_inputs = tokenizer(article,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
model_inputs

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

In [ ]:
raw_pred

In [ ]:
tokenizer.decode(raw_pred[0])

In [ ]:
tokenizer.decode(raw_pred[0])

In [ ]:
model.save_pretrained("/content")